In [129]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [119]:
def get_watches(page_count):
    
    watches_dict = {}

    try:
        for n in range(1, page_count+1):
            url = f"https://www.liveauctioneers.com/c/watches/97/?page={n}"

            response = requests.get(url)

            if response.status_code != 200:
                print(f"Failed to get the webpage: {response.status_code}")
                return []

            soup = BeautifulSoup(response.text, 'html.parser')

            watches_list = soup.find_all('section', class_ = "CategorySearchCard__CategorySearchCardGrid-sc-1o7izf2-1 dliokm")

            for watch in watches_list:
                
                name_element = watch.find('span', class_='hui-text-body-primary text-text-primary')
                if name_element:
                    watch_name = name_element.text
                else:
                    # Skip or log the missing name
                    print(f"Missing watch name on page {n}")
                    continue

                price_element = watch.find('span', class_ = 'FormattedCurrency__StyledFormattedCurrency-sc-1ugrxi1-0 cqnbDD')

                if price_element:
                    watch_price_str = price_element.text
                    if watch_price_str[0] != "$":
                        print(f"Not in USD in page {n}. Excluding result to avoid confusion")
                        continue 
                    else:
                        watch_price = float(watch_price_str.replace('$', '').replace(',', ''))
                else:
                    print(f"Missing watch price on page {n}")
                    continue
                
                watches_dict[watch_name] = watch_price
        
        return watches_dict
    
    except Exception as e:
        print(f"Error has occurred: {e}")
        return []

In [152]:
def watch_shortlist(page_count, target_price, plus_minus):
    watches = get_watches(page_count = page_count)
    # shortlist = {}
    shortlist_df = pd.DataFrame()

    price_lower_bound = max(0, target_price - plus_minus)
    price_upper_bound = target_price + plus_minus

    for key, value in watches.items():
        if value >= price_lower_bound and value <= price_upper_bound:
            row = pd.DataFrame({"Name": [key], "Price": [value]})
            shortlist_df = pd.concat([shortlist_df, row], ignore_index=True)
    
    shortlist_df = shortlist_df.sort_values(by=["Price"], ascending= True)
    
    return shortlist_df

In [158]:
watches = watch_shortlist(page_count=50, target_price=2000, plus_minus=100)

Missing watch price on page 2
Missing watch price on page 5
Not in USD in page 13. Excluding result to avoid confusion
Missing watch price on page 17
Not in USD in page 17. Excluding result to avoid confusion
Not in USD in page 18. Excluding result to avoid confusion
Not in USD in page 18. Excluding result to avoid confusion
Not in USD in page 24. Excluding result to avoid confusion
Missing watch price on page 25
Missing watch price on page 28
Not in USD in page 29. Excluding result to avoid confusion
Not in USD in page 29. Excluding result to avoid confusion
Not in USD in page 35. Excluding result to avoid confusion
Not in USD in page 35. Excluding result to avoid confusion
Not in USD in page 36. Excluding result to avoid confusion
Not in USD in page 36. Excluding result to avoid confusion
Not in USD in page 36. Excluding result to avoid confusion
Not in USD in page 39. Excluding result to avoid confusion
Not in USD in page 39. Excluding result to avoid confusion
Missing watch price o

In [159]:
watches

,Name,Price
0,Universal Geneve 14K Tri-Compax Chronograph Me...,1900.0
3,Piaget Vintage Polo 18k Yellow Gold Quartz Watch,1900.0
10,Pomellato 18K Yellow and White Gold 0.20ct Dia...,1900.0
8,"Rolex Rolex Milgauss Stainless Stell Me, Watch...",1900.0
11,Scarce E Howard 14 Gold Split Plate Open Face ...,1900.0
6,Rolex Stainless Steel Oyster Perpetual Datejust,2000.0
2,"ULYSSE NARDIN, 18K yellow gold and enamel chro...",2000.0
4,"A Rolex, eighteen karat gold and stainless ste...",2000.0
12,"Superb 18k Gold Quarter Repeater Pocket Watch,...",2000.0
7,"Vacheron Constantin, Art Deco white gold wrist...",2000.0
